<a href="https://colab.research.google.com/github/ria1994maitra/Docs/blob/main/Clause_to_Tree_Triplets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import stanza
import matplotlib.pyplot as plt
import networkx as nx

stanza.download('en')       # This downloads the English models for the neural pipeline
nlp_stanza = stanza.Pipeline('en',processors=['tokenize','pos','constituency'])


2022-10-04 16:00:18 INFO: Downloading default packages for language: en (English)...
2022-10-04 16:00:20 INFO: File exists: C:\Users\MehulGupta\stanza_resources\en\default.zip
2022-10-04 16:00:25 INFO: Finished downloading models and saved to C:\Users\MehulGupta\stanza_resources.


2022-10-04 16:00:26 INFO: Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

2022-10-04 16:00:26 INFO: Use device: cpu
2022-10-04 16:00:26 INFO: Loading: tokenize
2022-10-04 16:00:26 INFO: Loading: pos
2022-10-04 16:00:26 INFO: Loading: constituency
2022-10-04 16:00:27 INFO: Done loading processors!


In [ ]:
def iter_tree_stanza(sent, depth): #recursive DFS
    a_inter = []
    if (sent.is_preterminal() == False) or (sent is None):
        for child in sent.children:
            # if str(' '.join(child.leaf_labels())).strip() in (',','(',')','i','ii','iii'):
            #     pass

            # if child.label in ('CC'):
            #   a_inter.append([depth , ' '.join(child.leaf_labels()) , child.label , '--'])

            if sent.label in ('S','VP','PP') and child.label in ('NP',):
                check = 0 

                for gchild in child.children:
                    if gchild.label in ('NP','VP','S'):
                        check = 1 

                if check == 1:
                    for item in iter_tree_stanza(child, depth+1):
                        a_inter.append(item)
                else: 
                    a_inter.append([depth , ' '.join(child.leaf_labels()) , child.label , 'Node'])


            elif sent.label in ('NP') and child.label in ('NP','NNP','NN','NML','NNS','JJ','VBN','ADJP'):
                check = 0 

                for gchild in child.children:
                    if gchild.label in ('NP','VP','NML','NN'):
                        check = 1 

                if check == 1:
                    for item in iter_tree_stanza(child, depth+1):
                        a_inter.append(item)
                else:
                    a_inter.append([depth , ' '.join(child.leaf_labels()) , child.label , 'Node'])                      
                                                        

            elif sent.label in ('NML') and child.label in ('NN'):
                a_inter.append([depth , ' '.join(child.leaf_labels()) , child.label , 'Node'])


            elif sent.label in ('VP') and child.label in ('VBZ','MD','VB','VBG','VBN','NNS','RB'):

                a_inter.append([depth , ' '.join(child.leaf_labels()) , child.label , 'Edge'])         


            elif sent.label in ('PP') and child.label in ('IN'):
                a_inter.append([depth , ' '.join(child.leaf_labels()) , child.label , 'Edge'])                                  
          
                    

                                                                                                   
    #####################
    # New Rule added here - End 
    #####################                   
            else:
              for item in iter_tree_stanza(child, depth+1):
                    a_inter.append(item)
    else:
        pass
    
    return a_inter

In [ ]:
def components_to_tree(results):
    # len_results = len(results)

    # for elmnt_seq in range(len_results):
    #     if results[elmnt_seq][2] == 'CC':
    #         if results[elmnt_seq-1][3] == results[elmnt_seq+1][3]:
    #             results[elmnt_seq][3] = results[elmnt_seq-1][3]

    ### Combine successsive edge
    elmnt_seq = 0
    while elmnt_seq < len(results)-1:
        if results[elmnt_seq][3] == 'Edge' and results[elmnt_seq+1][3] == 'Edge':
            results[elmnt_seq][1] = results[elmnt_seq][1] + ' ' +  results[elmnt_seq+1][1]
            results.remove(results[elmnt_seq+1])
            elmnt_seq=elmnt_seq-1
        else:
            elmnt_seq = elmnt_seq + 1

    ### Combine successsive Nouns / Adjectives
    elmnt_seq = 0
    while elmnt_seq < len(results)-1:
        if (results[elmnt_seq][2] in ('NN','NNS') and results[elmnt_seq+1][2] in ('NN','NNS')) or (results[elmnt_seq][2] in ('JJ','JJR','JJS','ADJP') and results[elmnt_seq+1][2] in ('JJ','JJR','JJS','ADJP')):
            results[elmnt_seq][1] = results[elmnt_seq][1] + ' ' +  results[elmnt_seq+1][1]
            results.remove(results[elmnt_seq+1])
            elmnt_seq=elmnt_seq-1
        else:
            elmnt_seq = elmnt_seq + 1     

    ### Combine Adjectives to entity
    elmnt_seq = 0
    while elmnt_seq < len(results)-1:
        if results[elmnt_seq][2] in ('JJ','JJR','JJS','ADJP'):
            results[elmnt_seq][1] = results[elmnt_seq][1] + ' ' +  results[elmnt_seq+1][1]
            results[elmnt_seq][2] = results[elmnt_seq+1][2]
            results.remove(results[elmnt_seq+1])
        elmnt_seq = elmnt_seq + 1    


    ### Combine 's based strings
    elmnt_seq = 0
    while elmnt_seq < len(results)-1:
        if "'s" in results[elmnt_seq][1]:
            results[elmnt_seq][1] = results[elmnt_seq][1] + ' ' +  results[elmnt_seq+1][1]
            results.remove(results[elmnt_seq+1])
        elmnt_seq = elmnt_seq + 1     


    ### Combine "any" , "of" and whatever is followed up entity 
    elmnt_seq = 0
    while elmnt_seq < len(results)-1:
        if results[elmnt_seq][1].strip().lower() == 'any' and results[elmnt_seq+1][1].strip().lower() == 'of':
            results[elmnt_seq][1] = results[elmnt_seq][1] + ' ' +  results[elmnt_seq+1][1] + ' ' +  results[elmnt_seq+2][1]
            results.remove(results[elmnt_seq+2])
            results.remove(results[elmnt_seq+1])
        elmnt_seq = elmnt_seq + 1 


    ### Combine if starting is an edge, to create node
    if results[0][3] == 'Edge':
        new_node_val = results[0][1]
        elmnt_seq = 1
        while elmnt_seq < len(results)-1:
            if results[elmnt_seq][3] == 'Node':
                new_node_val = new_node_val + ' ' +  results[elmnt_seq][1]
                results.remove(results[elmnt_seq])
            else:
                break
        results[0][3] = 'Node'
        results[0][1] = new_node_val

    # return results

    ### Convert result into tree
    elmnt_seq = 0
    prev_node = [results[elmnt_seq][1]]
    prev_edge = results[elmnt_seq+1][1]
    tree_triplets = []
    current_node_series = []

    for elmnt_seq in range(2,len(results)):

        if results[elmnt_seq][3] == 'Edge':
            prev_edge = results[elmnt_seq][1]
            prev_node = current_node_series
            current_node_series = []

        else:
            if elmnt_seq+1 < len(results):
                if results[elmnt_seq+1][3] == 'Edge':

                    if results[elmnt_seq-1][3] == 'Edge':
                        for nodes in prev_node:
                            tree_triplets.append([nodes,prev_edge,results[elmnt_seq][1]])
                        current_node_series = [results[elmnt_seq][1]]
                    
                    else:
                        for nodes in prev_node:
                            tree_triplets.append([nodes,prev_edge,results[elmnt_seq][1]])
                        current_node_series.append(results[elmnt_seq][1])
            
                else:
                    if results[elmnt_seq-1][3] == 'Edge':
                        for nodes in prev_node:
                            tree_triplets.append([nodes,prev_edge,results[elmnt_seq][1]])
                        current_node_series.append(results[elmnt_seq][1])
                    
                    else:
                        for nodes in prev_node:
                            tree_triplets.append([nodes,prev_edge,results[elmnt_seq][1]])
                        current_node_series.append(results[elmnt_seq][1])

            else:
                for nodes in prev_node:
                    tree_triplets.append([nodes,prev_edge,results[elmnt_seq][1]])



    return tree_triplets

In [ ]:
clause_list = [
              "Customer agrees that CyberSource and its Affiliates may access and use Personal Information for improving/enhancing underlying products used by Customer, detecting data security incidents, or improving and enhancing security and fraud prevention tools for use by Customer and/or any other customers and clients of CyberSource or its Affiliates",
              # "Issuer hereby consents to Visa or its affiliate providing Cardinal with access to Issuer's fraud, chargeback and authentication- and authorization-level data for Visa transactions, solely to enable Cardinal to provide reporting to Issuer and for analyses and efforts to improve and enhance the fraud and risk capabilities of Cardinal's Service.",
              # "Cardinal may, however, independently use anonymous and/or aggregated information for pusposes of improving its products and services in accordance with applicable law.",
              # "CyberSource fruther represents and warrants that CyberSource will not use or disclose unique, non-public end-user data submitted by customer except as reasonably necessary (i) to provide the transaction services to customer hereunder, (ii) to provide fraud screen services generally without disclosing personally identifiable end-user information ,or, (iii) as otherwise permitted or required by law",
              # "For clarity, the foregoing consent does not authorize Visa to sell merchant's data or disclose it to third parties without merchant's prior written consent.",
              # "The rights, duties and/or obligations of CyberSource under this agreement may be exercised and/or performed by CyberSource and/or any of CyberSource's affiliates, or any of their subcontractors and/or agents.",
              ## Sample from online
              # "The employment relationship between the parties shall be governed by this Agreement and by the policies and practices established by the Company and/or its Board.",
              # "SUBRECIPIENT shall also comply with all applicable parts of COUNTY’S WIOA Policies and Procedures for recruitment, intake, assessment and referral, copies of which are available from COUNTY’S PROJECT MANAGER.",
              ]


for clause in clause_list:
    doc3 = nlp_stanza(clause)
    sent3 = doc3.sentences[0].constituency.children[0]
    # print(sent3.label)
    # print(' '.join(sent3.leaf_labels()))
    results_stp1 = iter_tree_stanza(sent3, 0)

    # for items in results_stp1:
    #   print(items)

    # print("\n\n ----------------------------- \n\n")

    results = components_to_tree(results_stp1)

    for items in results:
        print(items)

    print("\n\n ----------------------------- ----------------------------- ----------------------------- \n\n")          
    

['Customer', 'agrees', 'CyberSource']
['Customer', 'agrees', 'its Affiliates']
['CyberSource', 'may access use', 'Personal Information']
['its Affiliates', 'may access use', 'Personal Information']
['Personal Information', 'for improving enhancing underlying products used by', 'Customer']
['Customer', 'detecting', 'data security incidents']
['data security incidents', 'improving enhancing', 'security fraud prevention tools']
['security fraud prevention tools', 'for', 'use']
['use', 'by', 'Customer']
['use', 'by', 'any other customers']
['use', 'by', 'clients']
['Customer', 'of', 'CyberSource']
['any other customers', 'of', 'CyberSource']
['clients', 'of', 'CyberSource']
['Customer', 'of', 'its Affiliates']
['any other customers', 'of', 'its Affiliates']
['clients', 'of', 'its Affiliates']


 ----------------------------- ----------------------------- ----------------------------- 




In [ ]:

edges = [["start",results[0][0]]]

for elements in results:
    edges.append([elements[0],elements[2]])
edge_lables = {("start",results[0][0]):"here"}



for elements in results:
    edge_lables[(elements[0],elements[2])] = elements[1]


G = nx.MultiDiGraph()
G.add_edges_from(edges)

color_map = []
for node in G:
    if node != "start":
        color_map.append('pink')
    else: 
        color_map.append("tab:blue")

pos = nx.spring_layout(G)
plt.figure()
nx.draw(
    G, pos, edge_color='black', width=1, linewidths=1,
    node_size=500, node_color=color_map, alpha=0.9,connectionstyle='arc3, rad = 0.1',
    labels={node: node for node in G.nodes()}
)
nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=edge_lables,
    font_color='red'
)


    


plt.axis('off')
# plt.figure(1,figsize=(1000,1000), dpi=1000)

fig = plt.figure()
 
fig.set_figheight(10)
fig.set_figwidth(10)

plt.show()

In [ ]:
pdf_info.render_to()

In [1]:
! pip install pypdfium2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 32.1 MB/s eta 0:00:00


In [5]:
import pypdfium2 as pdf

In [ ]:
pdf.PdfDocument.render_to(pdf.BitmapConv.pil_image, page_indices , scale = )
def render_to(converter, page_indices=None, n_processes=os.cpu_count(), **kwargs)

render_to: (self: PdfDocument, converter: Any, page_indices: Any | None = None, n_processes: int | None = os.cpu_count(), **kwargs: Any) -> Any

Concurrently render multiple pages, using a process pool executor.

If rendering only a single page, the call is simply forwarded to .PdfPage.render_to as a shortcut.

Parameters:
    page_indices (typing.Sequence[int] | None):
        A sequence of zero-based indices of the pages to render. Reverse indexing or duplicate page indices are prohibited.
If None, all pages will be included. The order of results is guaranteed to match the order of given page indices.
    n_processes (int):
        Target number of parallel processes.
    kwargs (dict):
        Keyword arguments to the renderer. See .PdfPage.render_to / .PdfPage.render_base.

Yields:
    typing.Any: Implementation-specific result object.


Get the bitmap as PIL image


In [ ]:
# SPDX-FileCopyrightText: 2022 geisserml <geisserml@gmail.com>
# SPDX-License-Identifier: Apache-2.0 OR BSD-3-Clause

from pypdfium2._helpers.misc import BitmapStrReverseToRegular

try:
    import PIL.Image
except ImportError:
    PIL = None

try:
    import numpy.ctypeslib
except ImportError:
    numpy = None


class BitmapConvBase:
    """
    Parent class for bitmap converters compatible with :meth:`.PdfPage.render_to` / :meth:`.PdfDocument.render_to`.
    The initialiser captures any arguments and adds them to the :meth:`.run` call.
    """
    
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs
    
    @staticmethod
    def run(result, renderer_kws, *args, **kwargs):
        """
        The actual converter function, to be implemented by the inheriting class.
        
        Parameters:
            result (tuple):
                Result of the :meth:`~.PdfPage.render_base` call (ctypes array, colour format, size).
            renderer_kws (dict):
                Dictionary of rendering keywords that were passed in by the caller.
            args (tuple):
                Further positional arguments to the converter, as captured by the initialiser.
            kwargs (dict):
                Further keyword arguments to the converter, as captured by the initialiser.
        Returns:
            typing.Any: The converted rendering result (implementation-specific).
        """
        raise NotImplementedError("Inheriting class must provide run() method.")


class BitmapConv:
    """
    Built-in converters to be applied on the rendering result.
    """
    
    class any (BitmapConvBase):
        """
        Simple factory for converters that merely work with the ctypes array, while passing through additional information unaffected.
        
        Example:
            ``render_to(BitmapConv.any(bytes), **kwargs)``:
                Get an independent copy of the pixel data as bytes.
        
        Parameters:
            converter (typing.Callable):
                A function to translate a ctypes array to a different data type.
        Returns:
            (typing.Any, str, (int, int)): The converted bitmap (implementation-specific), and additional information returned by :meth:`~.PdfPage.render_base` (colour format, size).
        """
        
        @staticmethod
        def run(result, renderer_kws, converter):
            c_array, cl_format, size = result
            return converter(c_array), cl_format, size
    
    
    class numpy_ndarray (BitmapConvBase):
        """
        *Requires* :mod:`numpy`
        
        Get the bitmap as shaped NumPy array referencing the original ctypes array.
        This converter never makes a copy of the data.
        
        Returns:
            (numpy.ndarray, str): NumPy array, and colour format.
        """
        
        @staticmethod
        def run(result, renderer_kws):
            
            if numpy is None:
                raise RuntimeError("NumPy library needs to be installed for numpy_ndarray() converter.")
            
            c_array, cl_format, (width, height) = result
            np_array = numpy.ctypeslib.as_array(c_array)
            np_array.shape = (height, width, len(cl_format))
            
            return np_array, cl_format
    
    # PIL stands for Python Imaging Library, and it's the original library that enabled Python to deal with images.
    class pil_image (BitmapConvBase):
        """
        *Requires* :mod:`PIL`
        
        Get the bitmap as PIL image.
        
        Parameters:
            prefer_la (bool):
                If :data:`True`, automatically convert ``RGBA``/``BGRA`` to ``LA`` if rendering in greyscale mode with alpha channel
                (PDFium does not provide ``LA`` output directly).
        Returns:
            PIL.Image.Image: The image object.
        
        Hint:
            This uses :func:`PIL.Image.frombuffer` under the hood.
            If possible for the colour format in question, the image will reference the ctypes array. Otherwise, PIL may create a copy of the data.
            Among the pixel formats supported by PDFium, PIL can directly work with ``RGBA``, ``RGBX`` or ``L``.
            You may want to consider setting the rendering parameters *rev_byteorder* and *prefer_bgrx* to :data:`True` to generate natively compatible output.
        """
        
        @staticmethod
        def run(result, renderer_kws, prefer_la=False):
            
            if PIL is None:
                raise RuntimeError("Pillow library needs to be installed for pil_image() converter.")
            
            c_array, cl_src, size = result
            cl_dst = cl_src
            if cl_src in BitmapStrReverseToRegular.keys():
                cl_dst = BitmapStrReverseToRegular[cl_src]
            
            pil_image = PIL.Image.frombuffer(cl_dst, size, c_array, "raw", cl_src, 0, 1)
            if prefer_la:
                if renderer_kws.get("greyscale", False) and cl_dst == "RGBA":
                    pil_image = pil_image.convert("LA")
            
            return pil_image


class BitmapConvAliases:
    """
    Base class containing rendering target aliases.
    Currently retained for backwards compatibility, but may be deprecated in the future.
    Consider using the :meth:`.PdfPage.render_to` / :meth:`.PdfDocument.render_to` APIs instead.
    """
    
    def render_to(self):
        """ Method to be implemented by the inheriting class. """
        raise NotImplementedError("Inheriting class must provide render_to() method.")
    
    def render_tobytes(self, **kwargs):
        """ Alias for ``render_to(BitmapConv.any(bytes), ...)``. """
        return self.render_to(BitmapConv.any(bytes), **kwargs)
    
    def render_tonumpy(self, **kwargs):
        """ Alias for ``render_to(BitmapConv.numpy_ndarray, ...)``. """
        return self.render_to(BitmapConv.numpy_ndarray, **kwargs)
    
    def render_topil(self, prefer_la=False, **kwargs):
        """ Alias for ``render_to(BitmapConv.pil_image, ...)``. """
        return self.render_to(BitmapConv.pil_image(prefer_la=prefer_la), **kwargs)
